# Final Project - James Quacinella

Fo the final project, I will look at the follower network of one of the think tank Twitter account and perform clustering to find groups of associated accounts. Looking at the clusters, I hope to identify what joins them by performing some NLP tasks on the account's profile contents.

## Step 1 - Crawl Twitter for Followers

The next section of code does not run in the notebook, but is a copy of the crawler code created for this project. It will take a single account, get the first level followers, and then grab the 'second-level' followers. Those second level follower are only added if they were nodes in the first level (so we focus on the main account, not other accounts tangentially related).

In [ ]:
#import graphlab as gl
import pickle
import twitter
import logging
import time
from collections import defaultdict


### Setup a console and file logger

logger = logging.getLogger('crawler')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('crawler.log')
fh.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
fh.setFormatter(formatter)
logger.addHandler(ch)
logger.addHandler(fh)

### Setup signals to make sure API calls only take 60s at most

from functools import wraps
import errno
import os
import signal

class TimeoutError(Exception):
    pass

def timeout(seconds=60, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

@timeout()
def getFollowers(api, follower):
    ''' Function that will get a user's list of followers from an api object. 
    NOTE: the decorator ensures that this only runs for 60s at most. '''
    # return api.GetFollowerIDs(follower)
    return api.GetFriendIDs(follower)


### Twitter API

# Lets create our list of api OAuth parameters
API_TOKENS = [
    {"consumer_key": 'yp4wi4FASXbsRKa6JxYqzhUlH',
    "consumer_secret": 'Wkh1d5ygAOp4Bp65syFzHRN4xQsS8O4FvU3zHWosX8NXCqMpcl',
    "access_token_key": '16562593-F6lRFe7iyoQEahezhPmaI64oInHZD0LNpcIbbq7Wy',
    "access_token_secret": 'weregYL8n6DI7yZy9pkizIJ78rH2GY02Do9jvpTe7rCey',
    "requests_timeout": 60},

    {"consumer_key": 'NsNYFG9LtZV2XMyigPaCKVyVz',
    "consumer_secret": '4J1vlowybipqXnSrKgLBvmzPmwqx71uHN32noljTgDLS2xQNfI',
    "access_token_key": '16562593-NCuQWVnpzcnB55w7VLdoCkdobdUQBRDJKjIPXAksP',
    "access_token_secret": 'nX9OksrYQxj0jBXYJTkUjlX5mZh4rZljfVRXtSM3Tjc8c',
    "requests_timeout": 60},

    {"consumer_key": 'ZcAMGe2MUcnTO9ATCIo563SHN',
    "consumer_secret": 'dJAB7mBfoYyx27Yccbmzz98GtNigAA67Ish9Y1NjN2wNznciM1',
    "access_token_key": '16562593-AmaoKVLEYL3o8rVUS3b6u4PUbVPTI6BPsyaqCdwxY',
    "access_token_secret": '8pjYJCFWTErJlb2WSkLwsYNoptVazQQs95JAvIU8JApUA',
    "requests_timeout": 60},

    {"consumer_key": 'avZpjObqQN9vue2Y4gu9zIF9X',
    "consumer_secret": 'Ka6WCj3fyon5yGgf5YJIIl8nVcLcUh5YT99N58qy8qv4kfaMbc',
    "access_token_key": '16562593-VNuGD09Cr29ZlzNCWnV5MOujU7PsexSwfTgfKQNqC',
    "access_token_secret": '9P3hB3qDb9zPDFCUhWU16N4CMXPwHacl6HJbCc0EuGj7s',
    "requests_timeout": 60},

    {"consumer_key": 'sQ9H5NKteroNZSWvIrkSWvXR0',
    "consumer_secret": 'lC0ttZKdIZhhJAE1I5RxMxdjpSiADQCVUnHS7LbtfVmI2pz2F2',
    "access_token_key": '16562593-4LOk7QkXWD0boF01BmZ6NP2oPtHmDZ1OVJ883aANG',
    "access_token_secret": 'JJ85qMqzVowN1KdQ6w4YlhJB9YF9eWbw6SGbxQoU6gvne',
    "requests_timeout": 60},

    {"consumer_key": 'DHppZ2LG3iYj8vEx7ibRRLN35',
    "consumer_secret": 'wdTQeyp7ZNDN7ne40IriRw7Ah1J8cAi2OIlw4MVtgpq5MMKjYE',
    "access_token_key": '16562593-WN8zvEWAxVfJPrneMwUjDoVQw0geuLckOOJqFimsC',
    "access_token_secret": 'ZgVi2onPB3RPGtRmPBs6QXymIMgXwJHUOQycesp64S0Hp',
    "requests_timeout": 60},

    {"consumer_key": 'lIgtfdkC2WmN7XAcicrGygQBp',
    "consumer_secret": '2D9WIJN2MIPwFpMeIGcP6vWjQC8vvy7G5ZlHMSH1F1CsgWGKfz',
    "access_token_key": '16562593-7lhPpeZNNAGoQQJnqcnTtBiGq1O52XMZ4CMeVqXiY',
    "access_token_secret": 'WKRBQsr36MMB2EpCcZLr89ik0MSJfPoBORCKu9E1hw96I',
    "requests_timeout": 60},

    {"consumer_key": '1XFu2urZzoMoC5sadXAjA7IoQ',
    "consumer_secret": 'FrJOlHfNLp3M7ejJWiO5k74E9ai6L5EzQJ45HmlsUINbh8qUUi',
    "access_token_key": '16562593-Texko6g7VyCwhNUfxBDoJKJl4058hpvQkqAYWRKpi',
    "access_token_secret": 'ISZCTvN6bYJVaJ3Z2iidQObTzE2pxkINBLi0WWe9Ab2Zv',
    "requests_timeout": 60},

    {"consumer_key": 'r8Bvdm6I8QrRPuVzP4VtRYpqd',
    "consumer_secret": 'CzA8u8M8nDiDCCrSzCsXpR3SyTGCaLppDWbdTxSg78ZKgtKkhh',
    "access_token_key": '16562593-I3l0ZSmfZbMxIQ2NbiiM2eDMA4KNzFmFBeUkWxunR',
    "access_token_secret": '9HkILP4kSMF0hgvsB126jpoUzsRXETYMlSM0YSKb2yMJH',
    "requests_timeout": 60},

    {"consumer_key": 'NmMjfP1Zt3n2VDZ15X7SDGM6G',
    "consumer_secret": 'j9JBx7HUbMpcDnFteiIAAgHSoA8idlqQ20A1xbvnMrqMrOHQ1n',
    "access_token_key": '16562593-zUNyMUdO9JnSIstmTrqdyHHmX2lpv9NqkQxGC8faP',
    "access_token_secret": 'DEeHvLjTXlxNGmqDntXOK0cJCX08cnpg0btoRXWATW3X2',
    "requests_timeout": 60}
]

# Now create a list of twitter API objects
apis = []
for token in API_TOKENS:
    apis.append( twitter.Api(consumer_key=token['consumer_key'],
                    consumer_secret=token['consumer_secret'],
                    access_token_key=token['access_token_key'],
                    access_token_secret=token['access_token_secret'],
                    requests_timeout=60))


# The account id / screen name we want followers from
account_screen_name = 'fairmediawatch'
account_id = '54679731'

# Keep track of nodes connected to account, and all edges we need in the graph
nodes = set()
edges = defaultdict(set)


# Try to load first level followers from pickle;
# otherwise, generate them from a single API call and save via pickle
try:
    logger.info("Loading followers for %s" % account_screen_name)
    f = open("following1", "rb")
    following = pickle.load(f)
except Exception as e:
    logger.info("Failed. Generating followers for %s" % account_screen_name)
    following = api.GetFriendIDs(screen_name=account_screen_name)
    pickle.dump(following, open("following1", "wb"))

# Try to load the nodes and first level edges from pickle;
# otherwise generate them from the 'following' list and save
try:
    logger.info("Loading nodes and edges for depth = 1, for %s" % account_screen_name)
    n = open("nodes.follow1.set", "rb")
    e = open("edges.follow1.dict", "rb")
    nodes = pickle.load(n)
    edges = pickle.load(e)
except Exception as e:
    logger.info("Failed. Generating nodes and edges for depth = 1, for %s" % account_screen_name)
    for follower in following:
        nodes.add(follower)
        edges[account_id].add(follower)
    pickle.dump(nodes, open("nodes.follow1.set", "wb"))
    pickle.dump(edges, open("edges.follow1.dict", "wb"))



### Crawling for Depth2


# Index the api list, and start from the first api object
api_idx = 0
api = apis[api_idx]

# Some accounts give us issues (either too many followers or no permissions)
blacklist= [74323323, 43532023, 19608297, 25757924, 240369959, 173634807, 17008482, 142143804]
api_updated = False

# It is nice to start from a point in the list, instead of from the beginning
starting_point = 142143804
if starting_point:
    starting_point_idx = following.index(starting_point)
    following_iter = range(starting_point_idx, len(following))
else:
    following_iter = range(len(following))

# Try loading second layer of followers from pickle, otherwise start from scratch
try:
    f = open("edges.follow2.dict", "rb")
    edges = pickle.load(f)
    logger.info("Loaded edges.follow2 into memory!")
except Exception as e:
    logger.info("Starting from SCRATCH: did not load edges.follow2 into memory!")
    pass

# For each follower of the main account ...
for follower_idx in following_iter:
    follower = following[follower_idx]
    success = False
    
    # ... check if they are on the blacklist; if so, skip
    if follower in blacklist:
        logger.info("Skipping due to blacklist")
        continue

    # Otherwise, attempt to get list of their followers
    followers_depth2_list = []
    while not success:
        try:
            logger.info("Getting followers for follower %s" % follower)
            followers_depth2_list = getFollowers(api, follower)
            success = True
        except TimeoutError as e:
            # If api call takes too long, move on
            logger.info("Timeout after 60s for follower %d" % follower)
            success = True      # technically not a success but setting flag so next loop moves on
            continue
        except Exception as e:
            # IF we get here, then we hit API limits
            logger.info("API Exception %s; api-idx = %d" % (str(e), api_idx))
            
            # Are we at the begining of api list? 
            # IF so, dump edges so far via pickle and sleep
            if api_updated and api_idx % len(API_TOKENS) == 0 and api_idx >= len(API_TOKENS):
                logger.info("Save edges to pickle file for follower = %s" % follower)
                pickle.dump(edges, open("edges.follow2.dict", "wb"))
                logger.info("Sleeping ...")
                time.sleep(60)
                api_updated = False
            # Otherwise, move on to the next api object and try again
            else:
                api_idx += 1
                api = apis[api_idx % len(API_TOKENS)]
                api_updated = True
            
    
    # After getting the followers, find the intersection of those followers
    # with those of the first-level followers and add to edge dict
    if followers_depth2_list:
        logger.info("Adding followers to the graph")
        edges[follower].update(nodes.intersection(followers_depth2_list))


# Write out final list of edges via pickle
logger.info("Save edges to pickle file for follower = %s" % follower)
pickle.dump(edges, open("edges.follow2.dict", "wb"))

Instead of running the above, lets just load everything via pickle:

In [22]:
import pickle
n = open("nodes.follow1.set", "rb")
nodes = pickle.load(n)

e = open("edges.follow2.dict", "rb")
edges = pickle.load(e)

f = open("following1", "rb")
following = pickle.load(f)

## Step 2 - Generate Graph from Crawl

First, we generate CSV files so we can load data into GraphLab Create.

In [23]:
# Hide some silly output
import logging
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Import everything we need
import graphlab as gl

# Generate CSVs from the previous crawl
# TODO
f = open('vertices.csv', 'w')
f.write('id\n')
for node in nodes:
    f.write(str(node) + "\n")
f.close()

f = open('edges.csv', 'w')
f.write('src,dst,relation\n')
for node, followers in edges.iteritems():
    for follower in followers:
        f.write('%s,%s,%s\n' % (follower, node, 'follows'))
f.close()

Next, let us use these CSV files and load them into a graph object called g:

In [26]:
# Load Data
gvertices = gl.SFrame.read_csv('vertices.csv')
gedges = gl.SFrame.read_csv('edges.csv')

# Create graph
g = gl.SGraph()
g = g.add_vertices(vertices=gvertices, vid_field='id')
g = g.add_edges(edges=gedges, src_field='src', dst_field='dst')
g = g.add_edges(edges=gedges, src_field='dst', dst_field='src')

PROGRESS: Finished parsing file /home/james/Development/Masters/IndependentStudy/Final/vertices.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.02126 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/james/Development/Masters/IndependentStudy/Final/vertices.csv
PROGRESS: Parsing completed. Parsed 1106 lines in 0.020157 secs.
PROGRESS: Finished parsing file /home/james/Development/Masters/IndependentStudy/Final/edges.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.114567 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str]
If parsing fails due to incorrect types, you can correct
the

Lets try to visualize the graph!

In [28]:
# Visualize graph?
gl.canvas.set_target('browser')
g.show(vlabel="id")

Canvas is accessible via web browser at the URL: http://localhost:48677/index.html
Opening Canvas in default web browser.


Looks like its too large of a graph to display. Let's see if we can filter the graph by vertex degree and visualize the stronger connections:

In [34]:
pr = gl.pagerank.create(g)
pr.get('pagerank').topk(column_name='pagerank')

PROGRESS: Counting out degree
PROGRESS: Done counting out degree
PROGRESS: +-----------+-----------------------+
PROGRESS: | Iteration | L1 change in pagerank |
PROGRESS: +-----------+-----------------------+
PROGRESS: | 1         | 617.534               |
PROGRESS: | 2         | 135.25                |
PROGRESS: | 3         | 30.9247               |
PROGRESS: | 4         | 8.64859               |
PROGRESS: | 5         | 2.52531               |
PROGRESS: | 6         | 0.885368              |
PROGRESS: | 7         | 0.323184              |
PROGRESS: | 8         | 0.126578              |
PROGRESS: | 9         | 0.0503135             |
PROGRESS: | 10        | 0.0203128             |
PROGRESS: | 11        | 0.00825336            |
PROGRESS: +-----------+-----------------------+


__id,pagerank,delta
54679731,7.15893054698,2.0816305339e-05
59159771,5.73589508502,5.14342970437e-06
169182727,5.68248985863,2.48873868332e-05
16935292,4.98957011223,3.37281975531e-05
1947301,4.39339614539,1.60673869321e-06
23839835,4.36113011846,1.93642112576e-05
16076032,4.34163894719,7.78788532152e-06
10117892,3.96520683672,4.1042595571e-06
16955991,3.84512060914,1.25983857795e-05
478203018,3.40106898918,2.55084466292e-05


In [36]:
pr.summary()

Class                                   : PagerankModel

Graph
-----
num_edges                               : 210012
num_vertices                            : 1107

Results
-------
graph                                   : SGraph. See m['graph']
change in last iteration (L1 norm)      : 0.0083
vertex pagerank                         : SFrame. See m['pagerank']

Settings
--------
maximun number of iterations            : 20
convergence threshold (L1 norm)         : 0.01
probablity of random jumps to any node in the graph: 0.15

Metrics
-------
training time (secs)                    : 0.2171
number of iterations                    : 11

Queryable Fields
----------------
training_time                           : Total training time of the model
graph                                   : A new SGraph with the pagerank as a vertex property
delta                                   : Change in pagerank for the last iteration in L1 norm
reset_probability                       : The probablity 